## Oracle Cloud Instance POC


[https://porter.io/github.com/Yelp/bravado]

## Example with Basic Authentication


```
from bravado.requests_client import RequestsClient
from bravado.client import SwaggerClient

http_client = RequestsClient()
http_client.set_basic_auth(
    'api.yourhost.com',
    'username', 'password'
)
client = SwaggerClient.from_url(
    'http://petstore.swagger.io/v2/swagger.json',
    http_client=http_client,
)
pet = client.pet.getPetById(petId=42).response().result

```

## IaaS Compute Authentication

API calls to Compute Classic require basic authentication (user name and password). **You can pass your username and password with every API call or you can pass a valid authentication token**. To get a valid authentication token, send an HTTP request to authenticate the user credentials. If the authentication request succeeds, the server returns a cookie containing an authentication token that is valid for 30 minutes. The client making the API calls must include this cookie in the API calls.

To request for an authentication token and store the authentication token in an environment variable:


In [1]:
import json

In [2]:
from bravado_core.formatter import SwaggerFormat  # noqa
from bravado_core.param import marshal_param
from bravado_core.spec import Spec

In [3]:
from bravado.requests_client import RequestsClient
from bravado.client import SwaggerClient
from bravado.swagger_model import load_file




In [4]:
# from secrets import opc_username, opc_password
import keyring

opc_username = "eric.harris@oracle.com"
opc_password = keyring.get_password(service_name="gc3@gc30003", username="eric.harris@oracle.com")
print(f"opc_username={opc_username}, opc_password={opc_password}")
rest_endpoint = "https://dbaas.oraclecloud.com/"

opc_username=eric.harris@oracle.com, opc_password=V@nadium123!


In [5]:
################################################################################
## Standard Library Imports
import sys, os
from decimal import Decimal
import re

################################################################################
## Third-Party Imports
import dateutil
import pytz
# from bson.int64 import long
from dataclasses import dataclass, field
from bravado_core.formatter import SwaggerFormat
from bravado_core.exception import SwaggerValidationError

################################################################################
## Project Imports
from numpy import long

from gc3_query.lib import *
from gc3_query.lib.gc3logging import get_logging
_debug, _info, _warning, _error, _critical = get_logging(name=__name__)

In [6]:

formats = []


def NO_OP(x):
    return x




DEFAULT_FORMATS = {
    'byte': SwaggerFormat(
        format='byte',
        to_wire=lambda b: b if isinstance(b, str) else str(b),
        to_python=lambda s: s if isinstance(s, str) else str(s),
        validate=NO_OP,  # jsonschema validates string
        description='Converts [wire]string:byte <=> python byte'),
    'date': SwaggerFormat(
        format='date',
        to_wire=lambda d: d.isoformat(),
        to_python=lambda d: dateutil.parser.parse(d).date(),
        validate=NO_OP,  # jsonschema validates date
        description='Converts [wire]string:date <=> python datetime.date'),
    # Python has no double. float is C's double in CPython
    'double': SwaggerFormat(
        format='double',
        to_wire=lambda d: d if isinstance(d, float) else float(d),
        to_python=lambda d: d if isinstance(d, float) else float(d),
        validate=NO_OP,  # jsonschema validates number
        description='Converts [wire]number:double <=> python float'),
    'date-time': SwaggerFormat(
        format='date-time',
        to_wire=lambda dt: (dt if dt.tzinfo else pytz.utc.localize(dt)).isoformat(),
        to_python=lambda dt: dateutil.parser.parse(dt),
        validate=NO_OP,  # jsonschema validates date-time
        description=(
            'Converts string:date-time <=> python datetime.datetime')),
    'float': SwaggerFormat(
        format='float',
        to_wire=lambda f: f if isinstance(f, float) else float(f),
        to_python=lambda f: f if isinstance(f, float) else float(f),
        validate=NO_OP,  # jsonschema validates number
        description='Converts [wire]number:float <=> python float'),
    'int32': SwaggerFormat(
        format='int32',
        to_wire=lambda i: i if isinstance(i, int) else int(i),
        to_python=lambda i: i if isinstance(i, int) else int(i),
        validate=NO_OP,  # jsonschema validates integer
        description='Converts [wire]integer:int32 <=> python int'),
    'int64': SwaggerFormat(
        format='int64',
        to_wire=lambda i: i if isinstance(i, long) else long(i),
        to_python=lambda i: i if isinstance(i, long) else long(i),
        validate=NO_OP,  # jsonschema validates integer
        description='Converts [wire]integer:int64 <=> python long'),
}



In [7]:
class BooleanString:
    str_to_bool: Dict[str, bool] = dict(true=True, false=False)

    def __init__(self, from_wire: str):
        self.from_wire= from_wire
        self._as_boolean = self.str_to_bool[from_wire.lower()]
        _debug(f'created')
        # if self.validate(boolish):
        #     self.as_boolean = self.boolean_string_values[boolish]
        # if self.boolish_literal != self.boolish:
        #     _warning(f"case sensitive data passed, self.boolish_literal={self.boolish_literal}")

    @classmethod
    def validate(cls, from_wire: str) -> bool:
        _debug(f"from_wire={from_wire}")
        try:
            as_boolean = cls.str_to_bool[from_wire.lower()]
        except KeyError:
            raise SwaggerValidationError(f"Value={from_wire} not recognized as BooleanString")
        return isinstance(as_boolean, bool)

    def __bool__(self):
        return self.as_boolean

    @property
    def as_boolean(self):
        as_boolean = self.str_to_bool[self.from_wire.lower()]
        return as_boolean

    @property
    def as_wire(self):
        return self.from_wire

    @classmethod
    def bool_to_wire(cls, b):
        _bool_to_wire = 'true' if b else 'false'
        return _bool_to_wire

    @classmethod
    def str_to_python(cls, s):
        _str_to_python = cls.str_to_bool.get(s, False)
        return _str_to_python


boolean_string_format = SwaggerFormat(
    # name of the format as used in the Swagger spec
    format='boolean_string',

    # Callable to convert a python object to_wire representations
    to_wire=lambda boolean_string_instance: boolean_string_instance.as_wire,

    # Callable to convert a from_wire to a python object
    to_python=lambda s: BooleanString(s),

    # Callable to validate the cidr in string form
    validate=BooleanString.validate,
    description='Converts "true" and "false" to/from equivalent booleans.'
)


formats.append(boolean_string_format)



In [8]:
rest_endpoint = 'https://dbaas.oraclecloud.com/'
swagger_file = 'dbcsServiceInstances.json'

config = {'validate_responses': True, 
          'validate_requests': True, 
#          'validate_swagger_spec': True, 
          'validate_swagger_spec': False, 
          'use_models': True, 
          'include_missing_properties': True,
          'default_type_to_object': True, 
          'internally_dereference_refs': False, 
          'formats': [boolean_string_format]}


In [9]:
from melddict import MeldDict
dbcs_spec_dict = json.loads(open(swagger_file, 'r').read())
dbcs_spec_dict.keys()


dict_keys(['swagger', 'info', 'schemes', 'paths', 'definitions'])

In [10]:
# Spec.from_dict(cls, spec_dict, origin_url=None, http_client=None, config=None):
dbcs_swagger_spec = Spec.from_dict(spec_dict=dbcs_spec_dict, origin_url=rest_endpoint , config=config)

ValueError: Duplicate "Request Body" model found at "https://dbaas.oraclecloud.com/#/definitions/create-instance". Original "Request Body" model at "https://dbaas.oraclecloud.com/#/definitions/change-role-postrequest"

In [ ]:
print(f"user_defined_formats=[{dbcs_swagger_spec.user_defined_formats}]")
print(dbcs_swagger_spec.origin_url)
print(dbcs_swagger_spec.api_url)

In [ ]:
print(dbcs_swagger_spec.config)


In [ ]:
http_client = RequestsClient()
http_client.set_basic_auth(rest_endpoint, opc_username, opc_password)



In [ ]:
# SwaggerClient.from_spec(spec_dict, origin_url=None, http_client=None, config=None)

In [ ]:
# client = SwaggerClient.from_url(
#     'http://petstore.swagger.io/v2/swagger.json',
#     http_client=http_client,
# )
# pet = client.pet.getPetById(petId=42).response().result


In [ ]:
auth_client = SwaggerClient.from_spec(spec_dict=auth_spec_dict, origin_url=rest_endpoint, http_client=http_client, config=config)

In [ ]:
print(f"user_defined_formats=[{auth_client.swagger_spec.user_defined_formats}]")
print(auth_client.swagger_spec.origin_url)
print(auth_client.swagger_spec.api_url)
print(f"bravado_config=[{auth_client.swagger_spec.config['bravado']}]")
print(f"schemes=[{auth_client.swagger_spec.client_spec_dict['schemes']}]")

In [ ]:
print(auth_client.swagger_spec.config)

In [ ]:
dir(auth_client)

In [ ]:
auth_client.Authenticate.POST_auth?

In [ ]:
body = f'{{"username": "{opc_username}", "password": "{opc_password}"}}'
print(body)

In [ ]:
auth_http_future = auth_client.Authenticate.POST_auth(Body=dict(username="eric.harris@oracle.com", password="V@nadium123!"))

In [ ]:
auth_http_future.future.request.url

In [ ]:
auth_response = auth_http_future.response()

In [ ]:
secrule_client = SwaggerClient.from_spec(spec_dict=spec_dict, origin_url=rest_endpoint, http_client=http_client, config=config)

In [ ]:
print(f"user_defined_formats=[{client.swagger_spec.user_defined_formats}]")
print(client.swagger_spec.origin_url)
print(client.swagger_spec.api_url)
print(f"bravado_config=[{client.swagger_spec.config['bravado']}]")
print(f"schemes=[{client.swagger_spec.client_spec_dict['schemes']}]")



In [ ]:
print(client.swagger_spec.config)

In [ ]:
dir(client.SecRules)

In [ ]:
config.keys()

```
service_operations = <bravado.client.ResourceDecorator object at 0x000001969CDF3BE0>
service_operation_name = 'getSecRule'
service_operation = getattr(service_operations, service_operation_name)
operation_headers={'Accept': 'application/oracle-compute-v3+json', 'Content-Type': 'application/oracle-compute-v3+json'}
partial_service_operation = partial(service_operation, _request_options={"headers": operation_headers})
p = functools.partial(<bravado.client.CallableOperation object at 0x000001969CDDAB00>, _request_options={'headers': {'Accept': 'application/oracle-compute-v3+json', 'Content-Type': 'application/oracle-compute-v3+json'}})

```


```
self.idm_root_container_name
Out[2]: 'Compute-587626604'
http_future = self.bravado_service_operations.listSecRule(container=self.idm_root_container_name)




self.idm_root_container_name
Out[2]: 'Compute-587626604'
request_url
Out[3]: 'https://compute.uscom-central-1.oraclecloud.com/secrule/Compute-587626604/'


```

In [ ]:
client.SecRules.listSecRule?

In [ ]:
http_future = client.SecRules.listSecRule(container='Compute-587626604', _request_options={'headers': {'Accept': 'application/oracle-compute-v3+json', 'Content-Type': 'application/oracle-compute-v3+json'}})

In [ ]:
request_url = http_future.future.request.url
print(request_url)

In [ ]:
service_response = http_future.response()

In [ ]:
http_client.authenticator